In [ ]:
import pandas as pd
import yfinance as yf
import pandas_datareader as pdr
import numpy as np
import matplotlib.pyplot as mplt
import seaborn as sn
import datetime
import streamlit as st
import os
import plotly.express as px
import plotly.colors as pcolors
import plotly.graph_objects as go
from scipy.optimize import minimize

In [ ]:
# --------------- lista de links e ideias --------------- #
# lista de tickers
# ver normalização dos precos das acoes , https://www.youtube.com/watch?v=FQp6kFER6v8
# https://mesmith027-streamlit-webapps-mc-pistreamlit-app-l4b15e.streamlit.app -> parece um infografico com bastante texto e gráficos legais

In [ ]:

os.chdir('C:/Users/Computadores Gamer/OneDrive/Área de Trabalho/python')
print(os.getcwd())

In [ ]:
# lista acoes
acoes = pd.read_csv('acoes-listadas.csv')['Código']
acoes.to_excel('acoes_listas.xlsx')

In [ ]:
yf.pdr_override() #corrige problemas da bibliotece do pandas_datareader

In [ ]:
acoes = pd.read_excel(os.path.join('C:/Users/Computadores Gamer/OneDrive/Área de Trabalho/python/acoes_listas.xlsx'))['Código']

acoes = acoes +'.SA'
acoes

In [172]:
# lista_acoes = ['AMER3.SA','PETR3.SA','VALE3.SA', 'CASH3.SA', 'ABEV3.SA']
data_i = '2021-01-01'
data_f = '2022-01-01'

acao_erro = []
tabela = pd.DataFrame()

for i in acoes:
    try:
        tabela[f'{i}'] = round(yf.download(i, start=data_i, end=data_f)['Adj Close'].resample('D').last(),2).dropna()
    except Exception as e:
        acao_erro.append(i)
        print('erro', acao_erro, e)
        tabela[f'{i}'] = pd.Series(dtype='float64')


# tabela_retorn1 = tabela.pct_change().dropna()

# media_retorno = tabela_retorn1.mean()
# # print(media_retorno)

# matriz_cov = tabela_retorn1.cov()
# # print(matriz_cov)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

C:\Users\Computadores Gamer\AppData\Local\Temp\ipykernel_3092\1413363126.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



[*********************100%***********************]  1 of 1 completed


C:\Users\Computadores Gamer\AppData\Local\Temp\ipykernel_3092\1413363126.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



[*********************100%***********************]  1 of 1 completed


...

C:\Users\Computadores Gamer\AppData\Local\Temp\ipykernel_3092\1413363126.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CTSA8.SA: No data found for this date range, symbol may be delisted
erro ['AURE3.SA', 'IGTI4.SA', 'IGTI4.SA', 'VIIA3.SA', 'ENBR3.SA', 'BOAS3.SA', 'MODL3.SA', 'CRDE3.SA', 'IGBR3.SA', 'PARD3.SA', 'WIZS3.SA', 'LLIS3.SA', 'BRML3.SA', 'DMMO3.SA', 'GETT3.SA', 'GETT4.SA', 'SULA4.SA', 'SULA3.SA', 'CEPE5.SA', 'TCNO4.SA', 'TCNO3.SA', 'CEPE6.SA', 'BKBR3.SA', 'MTIG4.SA', 'BLUT4.SA', 'BLUT3.SA', 'MODL4.SA', 'CARD3.SA', 'FNCN3.SA', 'LCAM3.SA', 'BIDI4.SA', 'BIDI3.SA', 'EEEL4.SA', 'EEEL3.SA', 'BBRK3.SA', 'CESP6.SA', 'CESP3.SA', 'CESP5.SA', 'MOSI3.SA', 'POWE3.SA', 'GNDI3.SA', 'LAME4.SA', 'LAME3.SA', 'OMGE3.SA', 'IGTA3.SA', 'JPSA3.SA', 'BRDT3.SA', 'JBDU4.SA', 'JBDU3.SA', 'HGTX3.SA', 'CCPR3.SA', 'DTEX3.SA', 'VVAR3.SA', 'PNVL4.SA', 'TESA3.SA', 'BTOW3.SA', 'LINX3.SA', 'BTTL3.SA', 'GPCP3.SA', 'GPCP4.SA', 'SMLS3.SA', 'MMXM3.SA', 'BSEV3.SA', 'CNTO3.SA', 'TIET4.SA', 'TIET3.SA', 'CEPE3.SA', 'CALI4.SA', 'BPAR3.SA', 'APTI4.

C:\Users\Computadores Gamer\AppData\Local\Temp\ipykernel_3092\1413363126.py:14: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



[*********************100%***********************]  1 of 1 completed


C:\Users\Computadores Gamer\AppData\Local\Temp\ipykernel_3092\1413363126.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [173]:
acao_erro
# acao_erro = pd.DataFrame(acao_erro)
# acao_erro.to_excel('acoes_erro.xlsx')

['AURE3.SA',
 'IGTI4.SA',
 'IGTI4.SA',
 'VIIA3.SA',
 'ENBR3.SA',
 'BOAS3.SA',
 'MODL3.SA',
 'CRDE3.SA',
 'IGBR3.SA',
 'PARD3.SA',
 'WIZS3.SA',
 'LLIS3.SA',
 'BRML3.SA',
 'DMMO3.SA',
 'GETT3.SA',
 'GETT4.SA',
 'SULA4.SA',
 'SULA3.SA',
 'CEPE5.SA',
 'TCNO4.SA',
 'TCNO3.SA',
 'CEPE6.SA',
 'BKBR3.SA',
 'MTIG4.SA',
 'BLUT4.SA',
 'BLUT3.SA',
 'MODL4.SA',
 'CARD3.SA',
 'FNCN3.SA',
 'LCAM3.SA',
 'BIDI4.SA',
 'BIDI3.SA',
 'EEEL4.SA',
 'EEEL3.SA',
 'BBRK3.SA',
 'CESP6.SA',
 'CESP3.SA',
 'CESP5.SA',
 'MOSI3.SA',
 'POWE3.SA',
 'GNDI3.SA',
 'LAME4.SA',
 'LAME3.SA',
 'OMGE3.SA',
 'IGTA3.SA',
 'JPSA3.SA',
 'BRDT3.SA',
 'JBDU4.SA',
 'JBDU3.SA',
 'HGTX3.SA',
 'CCPR3.SA',
 'DTEX3.SA',
 'VVAR3.SA',
 'PNVL4.SA',
 'TESA3.SA',
 'BTOW3.SA',
 'LINX3.SA',
 'BTTL3.SA',
 'GPCP3.SA',
 'GPCP4.SA',
 'SMLS3.SA',
 'MMXM3.SA',
 'BSEV3.SA',
 'CNTO3.SA',
 'TIET4.SA',
 'TIET3.SA',
 'CEPE3.SA',
 'CALI4.SA',
 'BPAR3.SA',
 'APTI4.SA',
 'CTSA8.SA']

In [ ]:
# tabelas pré-simulacao
numero_portfolios = 100000
tabela_retorn_esperados = np.zeros(numero_portfolios)
tabela_volatilidade_esperada = np.zeros(numero_portfolios)
tabela_sharpe = np.zeros(numero_portfolios)
tabela_pesos = np.zeros((numero_portfolios, len(lista_acoes))) # numero de linhas seria cada portfolio e o numero de colunas seria o peso de cada acoa em 'lista_acoes'


In [ ]:
# simulacao
for i in range(numero_portfolios):
    pesos_aleatorios = np.random.random(len(lista_acoes)) # gera pesos aleatorio
    pesos_aleatorios = pesos_aleatorios/np.sum(pesos_aleatorios) # descobre a porcentagem de cada peso sobre o total de pesos daquela linha
    tabela_pesos[i,:] = pesos_aleatorios # completa uma linha e pula pra próxima até o numero total de portfolios
    tabela_retorn_esperados[i] = np.sum(media_retorno * pesos_aleatorios * 252) 
    tabela_volatilidade_esperada[i] = np.sqrt(np.dot(pesos_aleatorios.T, np.dot(matriz_cov * 252, pesos_aleatorios))) # np.dot é o produto entre duas matrizes, linha x coluna
    tabela_sharpe[i] = tabela_retorn_esperados[i] / tabela_volatilidade_esperada[i]




In [ ]:
# achando a melhor carteira, analisando o indice de sharpe
indice_sharp_max = tabela_sharpe.argmax() # retorna o indice da linha onde esta o indice de sharpe maximo
melhor_carteira = tabela_pesos[indice_sharp_max] # com isso pego a carteira (um dos portfolios) que apresenta a melhor relação entre retorno e risco que seria o indice de sharpe
melhor_carteira
# aqui posso pensar em impor limites para cada papel

In [ ]:
tabela_retorn_esperados_arit = np.exp(tabela_retorn_esperados) - 1
tabela_retorn_esperados_arit

In [ ]:
fronteira_eficiente_y = np.linspace(tabela_retorn_esperados_arit.min(), tabela_retorn_esperados_arit.max(), 50 )   # defino range do meu eixo

def pegando_retorno (peso_teste):
    peso_teste = np.array(peso_teste)
    retorno = np.sum(media_retorno * peso_teste) * 252
    retorno = np.exp(retorno) - 1 # aqui estou passando os retornos para aritmeticos
    return retorno
    
def checando_soma_pesos(peso_teste):
    return np.sum(peso_teste)-1


def pegando_vol(peso_teste):
    peso_tste = np.array(peso_teste)
    vol = np.sqrt(np.dot(peso_teste.T, np.dot(matriz_cov * 252, peso_teste)))
    return vol

peso_inicial = [1/len(lista_acoes)] * len(lista_acoes)  # pesos iguais para todas as acoes
limites = tuple([(0,1) for i in lista_acoes])   # aqui nenhuma acao pode ter mais que 100%


eixo_x_fronteira = []

for retorno_possivel in fronteira_eficiente_y:
    restricoes = ({'type':'eq', 'fun':checando_soma_pesos}, {'type':'eq', 'fun' : lambda weight: pegando_retorno(weight) - retorno_possivel}) # é um dicionario de restricoes, quando a igualdade ('eq') for zero é pq a restricao fo satisfeita
    
    result = minimize(pegando_vol, peso_inicial, method='SLSQP', bounds = limites, constraints = restricoes)
    eixo_x_fronteira.append(result['fun'])
    


In [ ]:
fig, ax = mplt.subplots()
ax.scatter(tabela_volatilidade_esperada, tabela_retorn_esperados_arit, c=tabela_sharpe)
ax.scatter(tabela_volatilidade_esperada[indice_sharp_max], tabela_retorn_esperados_arit[indice_sharp_max], c = 'red')
ax.plot(eixo_x_fronteira, fronteira_eficiente_y)

mplt.show()


# ideia de plotar a carteira de menos risco
# fazer a linha reta que eu acho que é a linha de taxa livre de risco, CONFIRMAAAAAAAAAAAAR!!!!


In [ ]:
result